# ETL Project
### ***NFL Draft Analysis***

In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

##### Create secrets.py file with username and password to pgadmin

In [2]:
#Import username and password from secrets.py file
from secrets import username, password, database_name

# Extract

##### Extract the data from Wikipedia and  www.pro-football-reference.com. 
* Note: Since we are scraping data from tables we are using pandas otherwise we were going to need to use BeatifulSoup and Splinter

### Extract Draft Info from 2016-2020

In [3]:
#Assign Draft URL's to variable
draft_url_2016 = 'https://www.pro-football-reference.com/years/2016/draft.htm'


In [25]:
df.columns = df.columns.droplevel()
df

,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,...,Yds,TD,Rec,Yds,TD,Solo,Int,Sk,College/Univ,Unnamed: 28_level_1
0,1,1,LAR,Jared Goff,QB,21,2021,0,2,5,...,401,10,0,0,0,NaN,NaN,NaN,California,College Stats
1,1,2,PHI,Carson Wentz,QB,23,2021,0,1,6,...,1228,9,2,11,0,NaN,NaN,NaN,North Dakota St.,College Stats
2,1,3,SDG,Joey Bosa,DE,21,2021,0,3,6,...,0,0,0,0,0,205,NaN,56.0,Ohio St.,College Stats
3,1,4,DAL,Ezekiel Elliott,RB,21,2021,1,3,6,...,7194,54,282,2202,11,1,NaN,NaN,Ohio St.,College Stats
4,1,5,JAX,Jalen Ramsey,CB,21,2021,2,4,6,...,0,0,0,0,0,295,14,NaN,Florida St.,College Stats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,7,249,SFO,Prince Charles Iworah,CB,23,2016,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,Western Kentucky,College Stats
255,7,250,CLE,Scooby Wright III,ILB,22,2017,0,0,0,...,0,0,0,0,0,5,NaN,NaN,Arizona,College Stats
256,7,251,PHI,Joe Walker,ILB,23,2021,0,0,1,...,0,0,0,0,0,60,NaN,NaN,Oregon,College Stats
257,7,252,CAR,Beau Sandland,TE,23,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Montana St.,NaN


In [28]:
df1 = df[['Rnd','Pick','Player','Pos','Age','College/Univ']]
df1

,Rnd,Pick,Player,Pos,Age,College/Univ
0,1,1,Jared Goff,QB,21,California
1,1,2,Carson Wentz,QB,23,North Dakota St.
2,1,3,Joey Bosa,DE,21,Ohio St.
3,1,4,Ezekiel Elliott,RB,21,Ohio St.
4,1,5,Jalen Ramsey,CB,21,Florida St.
...,...,...,...,...,...,...
254,7,249,Prince Charles Iworah,CB,23,Western Kentucky
255,7,250,Scooby Wright III,ILB,22,Arizona
256,7,251,Joe Walker,ILB,23,Oregon
257,7,252,Beau Sandland,TE,23,Montana St.


In [68]:
years = (2016,2017,2018,2019,2020)
draft = pd.DataFrame()
for year in years:
    url = f'https://www.pro-football-reference.com/years/{year}/draft.htm'
    print(url)
    print('----------')
    read = pd.read_html(url)
    table = read[0]
    table.columns = table.columns.droplevel()
    table['year'] = year
    draft = draft.append(table, ignore_index=True)
draft

https://www.pro-football-reference.com/years/2016/draft.htm
----------
https://www.pro-football-reference.com/years/2017/draft.htm
----------
https://www.pro-football-reference.com/years/2018/draft.htm
----------
https://www.pro-football-reference.com/years/2019/draft.htm
----------
https://www.pro-football-reference.com/years/2020/draft.htm
----------


,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,...,TD,Rec,Yds,TD,Solo,Int,Sk,College/Univ,Unnamed: 28_level_1,year
0,1,1,LAR,Jared Goff,QB,21,2021,0,2,5,...,10,0,0,0,NaN,NaN,NaN,California,College Stats,2016
1,1,2,PHI,Carson Wentz,QB,23,2021,0,1,6,...,9,2,11,0,NaN,NaN,NaN,North Dakota St.,College Stats,2016
2,1,3,SDG,Joey Bosa,DE,21,2021,0,3,6,...,0,0,0,0,205,NaN,56.0,Ohio St.,College Stats,2016
3,1,4,DAL,Ezekiel Elliott,RB,21,2021,1,3,6,...,54,282,2202,11,1,NaN,NaN,Ohio St.,College Stats,2016
4,1,5,JAX,Jalen Ramsey,CB,21,2021,2,4,6,...,0,0,0,0,295,14,NaN,Florida St.,College Stats,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,7,251,SEA,Stephen Sullivan,TE,23,2020,0,0,0,...,0,0,0,0,NaN,NaN,NaN,LSU,College Stats,2020
1297,7,252,DEN,Tyrie Cleveland,WR,22,2021,0,0,0,...,0,6,63,0,4,NaN,NaN,Florida,College Stats,2020
1298,7,253,MIN,Kyle Hinton,G,22,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washburn,NaN,2020
1299,7,254,DEN,Derrek Tuszka,LB,24,2021,0,0,0,...,0,0,0,0,9,NaN,NaN,North Dakota St.,NaN,2020


# Transform

In [69]:
# Select the columns we are going to use
df = draft[['Rnd','Pick','Player','Pos','Age','College/Univ', 'year']]
df

,Rnd,Pick,Player,Pos,Age,College/Univ,year
0,1,1,Jared Goff,QB,21,California,2016
1,1,2,Carson Wentz,QB,23,North Dakota St.,2016
2,1,3,Joey Bosa,DE,21,Ohio St.,2016
3,1,4,Ezekiel Elliott,RB,21,Ohio St.,2016
4,1,5,Jalen Ramsey,CB,21,Florida St.,2016
...,...,...,...,...,...,...,...
1296,7,251,Stephen Sullivan,TE,23,LSU,2020
1297,7,252,Tyrie Cleveland,WR,22,Florida,2020
1298,7,253,Kyle Hinton,G,22,Washburn,2020
1299,7,254,Derrek Tuszka,LB,24,North Dakota St.,2020


In [70]:
# Save to csv to get the index to use it as ID
df.to_csv('../resources/nfl_draft2016-2020')

In [76]:
# read the csv to get first column and use it as id
draft = pd.read_csv('../resources/nfl_draft2016-2020')
draft

,Unnamed: 0,Rnd,Pick,Player,Pos,Age,College/Univ,year
0,0,1,1,Jared Goff,QB,21,California,2016
1,1,1,2,Carson Wentz,QB,23,North Dakota St.,2016
2,2,1,3,Joey Bosa,DE,21,Ohio St.,2016
3,3,1,4,Ezekiel Elliott,RB,21,Ohio St.,2016
4,4,1,5,Jalen Ramsey,CB,21,Florida St.,2016
...,...,...,...,...,...,...,...,...
1296,1296,7,251,Stephen Sullivan,TE,23,LSU,2020
1297,1297,7,252,Tyrie Cleveland,WR,22,Florida,2020
1298,1298,7,253,Kyle Hinton,G,22,Washburn,2020
1299,1299,7,254,Derrek Tuszka,LB,24,North Dakota St.,2020


In [89]:
# rename first column as player_id
draft = draft.rename(columns={'Unnamed: 0':'player_id','College/Univ': 'college'})
draft

,player_id,Rnd,Pick,Player,Pos,Age,college,year
0,100,1,1,Jared Goff,QB,21,California,2016
1,101,1,2,Carson Wentz,QB,23,North Dakota St.,2016
2,102,1,3,Joey Bosa,DE,21,Ohio St.,2016
3,103,1,4,Ezekiel Elliott,RB,21,Ohio St.,2016
4,104,1,5,Jalen Ramsey,CB,21,Florida St.,2016
...,...,...,...,...,...,...,...,...
1296,1396,7,251,Stephen Sullivan,TE,23,LSU,2020
1297,1397,7,252,Tyrie Cleveland,WR,22,Florida,2020
1298,1398,7,253,Kyle Hinton,G,22,Washburn,2020
1299,1399,7,254,Derrek Tuszka,LB,24,North Dakota St.,2020


In [78]:
# add 100 to the id so it has at least 3 numbers
draft['player_id'] = draft['player_id']+100
draft['player_id'] = draft['player_id']
draft

,player_id,Rnd,Pick,Player,Pos,Age,College/Univ,year
0,100,1,1,Jared Goff,QB,21,California,2016
1,101,1,2,Carson Wentz,QB,23,North Dakota St.,2016
2,102,1,3,Joey Bosa,DE,21,Ohio St.,2016
3,103,1,4,Ezekiel Elliott,RB,21,Ohio St.,2016
4,104,1,5,Jalen Ramsey,CB,21,Florida St.,2016
...,...,...,...,...,...,...,...,...
1296,1396,7,251,Stephen Sullivan,TE,23,LSU,2020
1297,1397,7,252,Tyrie Cleveland,WR,22,Florida,2020
1298,1398,7,253,Kyle Hinton,G,22,Washburn,2020
1299,1399,7,254,Derrek Tuszka,LB,24,North Dakota St.,2020


In [92]:
draft.loc[draft['college']=='California']

,player_id,Rnd,Pick,Player,Pos,Age,college,year
0,100,1,1,Jared Goff,QB,21,California,2016
166,266,5,163,Trevor Davis,WR,23,California,2016
242,342,7,237,Daniel Lasco,RB,23,California,2016
248,348,7,243,Kenny Lawler,WR,22,California,2016
347,447,3,87,Davis Webb,QB,22,California,2017
402,502,4,141,Chad Hansen,WR,22,California,2017
505,605,7,241,Khalfani Muhammad,RB,22,California,2017
748,848,7,225,Devante Downs,LB,22,California,2018
755,855,7,232,James Looney,DE,23,California,2018
1109,1209,3,68,Ashtyn Davis,S,23,California,2020


In [99]:
col = pd.read_csv('../resources/world_universities.csv')
col

,Unnamed: 0,institution
0,0,360 Degrees Beauty Academy
1,1,4-States Academy of Cosmetology
2,2,4-States Okmulgee Academy of Cosmetology
3,3,5 STAR TRAVEL CAREERS
4,4,A & W Healthcare Educators
...,...,...
12571,12571,Zhejiang University
12572,12572,Zhejiang University of Technology
12573,12573,Zhengzhou University
12574,12574,Zion Bible College


In [102]:
col.loc[col['institution']!= 'Berkeley' ]  

,Unnamed: 0,institution
0,0,360 Degrees Beauty Academy
1,1,4-States Academy of Cosmetology
2,2,4-States Okmulgee Academy of Cosmetology
3,3,5 STAR TRAVEL CAREERS
4,4,A & W Healthcare Educators
...,...,...
12571,12571,Zhejiang University
12572,12572,Zhejiang University of Technology
12573,12573,Zhengzhou University
12574,12574,Zion Bible College


In [86]:
# save to csv


# draft.to_csv('../resources/nfl_draft_id', index=False)

# Load